In [2]:
import requests
import zipfile
import os

In [3]:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
package_id = "bike-share-toronto-ridership-data"

# Get package information
package_url = f"{base_url}/api/3/action/package_show"
params = {"id": package_id}
package = requests.get(package_url, params=params).json()

Download

In [5]:
# Define folder where files will be saved
data_folder = "./data"
data_year = "2022"

# Iterate through resources and download matching files
for resource in package["result"]["resources"]:
    if not resource["datastore_active"]:
        resource_id = resource["id"]
        resource_name = resource["name"]
        resource_url = resource["url"]

        if f'bikeshare-ridership-{data_year}' in resource_name:  # Check for specific file name
            # Check and create folder if not exist
            if not os.path.exists(data_folder):
                os.makedirs(data_folder)
                print(f"Folder created: {data_folder}")

            response = requests.get(resource_url)
            if response.status_code == 200:
                file_extension = resource["format"].lower()
                filename = f"{resource_name}.{file_extension}"
                file_path = os.path.join(data_folder, filename)  # Save inside the 'data' folder

                with open(file_path, "wb") as file:
                    file.write(response.content)
                print(f"Saved as: {file_path}\n")
            else:
                print(f"Failed to download {resource_name}\n")


Saved as: ./data\bikeshare-ridership-2022.zip



Extract

In [6]:
def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)  # Delete the zip file after extraction

def process_directory(directory):
    for root, dirs, files in os.walk(directory, topdown=False):  # Ensure directories are processed last
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith('.zip'):
                print(f"Extracting {file_path}")
                extract_zip(file_path, root)
        
        # Ensure only CSV files remain
        for file in os.listdir(root):
            file_path = os.path.join(root, file)
            if os.path.isdir(file_path):
                continue  # Skip directories
            if not file.endswith('.csv'):
                print(f"Removing non-CSV file: {file_path}")
                os.remove(file_path)

In [18]:
zip_file_path = f"./data/bikeshare-ridership-{data_year}.zip"
extract_location = f"./bikeshare-ridership-{data_year}"

# if not os.path.exists(extract_location):
#     os.makedirs(extract_location)

extract_zip(zip_file_path, data_folder)
process_directory(data_folder)

In [7]:
process_directory(f"{data_folder}/bikeshare-ridership-{data_year}")

Extracting ./data/bikeshare-ridership-2022\Bike share ridership 2022-11.zip
